We want to find a representative file that was updated for Quarkus which we can use for an example
Consider:  
* https://github.com/mathianasj/eap-coolstore-monolith/blob/quarkus-migration/src/main/java/com/redhat/coolstore/service/InventoryNotificationMDB.java
* https://github.com/mathianasj/eap-coolstore-monolith/blob/quarkus-migration/src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java

The larger idea we are working towards is:
1) Find a file which has been changed and get the original + diff
2) Find a similar file we need to update, and get the original of that file
3) Send a request to LLM and see how well it does with a patch
4) Compare it to the diff we have

Documentation for library: https://gitpython.readthedocs.io/en/stable/quickstart.html#trees-blobs

We want to learn how to generate a patch from a diff.
Reading API docs at: https://gitpython.readthedocs.io/en/stable/reference.html#module-git.diff

Assume 'create_patch' will help.

`create_patch – If True, the returned Diff contains a detailed patch that if applied makes the self to other. Patches are somewhat costly as blobs have to be read and diffed.`

Looking at this git history: 
* https://github.com/mathianasj/eap-coolstore-monolith/commits/quarkus-migration/src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
# Playing around to learn the library
from git import Repo

example_git_path = "../data/coolstuff-quarkus"
example_file = "src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java"

# We want to find the commits associated to a specific file in the Git Repo
repo = Repo.init(example_git_path)
commits_for_file_generator = repo.iter_commits(all=True, max_count=10, paths=example_file)
commits_for_file = list(commits_for_file_generator)
print(f"{example_file} has {len(commits_for_file)} commits associated")
print(f"{commits_for_file}")

src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java has 2 commits associated
[<git.Commit "9fd5a29585234b32cdd81cea71d71408760c489b">, <git.Commit "6322db01226ef4a561244ce90a5df946b163f7ba">]


In our example for the given file:
* latest commit is in commits_for_file[0]
* original commit is in commits_for_file[1]

In [3]:
print(f"The last commit on the file `{example_file}` is `{commits_for_file[0]}`")
print(commits_for_file[0].stats.files[example_file])

The last commit on the file `src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java` is `9fd5a29585234b32cdd81cea71d71408760c489b`
{'insertions': 22, 'deletions': 33, 'lines': 55}


In [4]:
print(f"The original commit on the file `{example_file}` is `{commits_for_file[1]}`")
print(commits_for_file[1].stats.files[example_file])

The original commit on the file `src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java` is `6322db01226ef4a561244ce90a5df946b163f7ba`
{'insertions': 50, 'deletions': 0, 'lines': 50}


Lets look at the actual contents of the example file in git for it's latest commit

In [5]:
original_tree = repo.tree(commits_for_file[1])
updated_tree = repo.tree(commits_for_file[0])

blob = updated_tree[example_file]
print(f"Contents of {example_file} at HEAD")
print(f"\n---HEAD---")
print(blob.data_stream.read().decode())
print(f"\n-----\n")

Contents of src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java at HEAD

---HEAD---
package com.redhat.coolstore.service;

// import javax.ejb.ActivationConfigProperty;
// import javax.ejb.MessageDriven;
import jakarta.inject.Inject;
import jakarta.transaction.Transactional;

import org.eclipse.microprofile.reactive.messaging.Incoming;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;

import io.smallrye.reactive.messaging.annotations.Blocking;
import jakarta.enterprise.context.ApplicationScoped;

@ApplicationScoped
public class OrderServiceMDB {

	@Inject
	OrderService orderService;

	@Inject
	CatalogService catalogService;

	@Incoming("orders-incoming")
	@Blocking
	@Transactional
	public void onMessage(String orderStr) {
		System.out.println("\nMessage recd !");
		System.out.println("Received order: " + orderStr);
		Order order = Transformers.jsonToOrder(orderStr);
		System.out.println("Order object is " + order);
		orderService.sav

Exploring some error behaviors


In [6]:
from git import BadName 

try:
    repo.commit("bad_id")
except BadName as e:
    print(f"We caught Bad Name: {e}") 

We caught Bad Name: Ref 'bad_id' did not resolve to an object


We will create a class next we can reuse to get the diff for a given file

In [7]:
#| default_exp scm
#| export

from git import BadName, Repo

class GitDiff:
    def __init__(self, repo_path):
        self.repo_path = repo_path
        self.repo = Repo(self.repo_path) 

    def get_patch(self, start_commit_id, end_commit_id="HEAD"):
        # https://gitpython.readthedocs.io/en/stable/reference.html#git.diff.DiffIndex

        # If either commit_id is not valid, this will raise a BadName exception
        start_commit = self.repo.commit(start_commit_id)
        #end_commit = self.repo.commit(end_commit_id)
        return start_commit.diff(end_commit_id, create_patch=True) 
        
    def get_patch_for_file(self, start_commit_id, end_commit_id, file_path):
        diff_indexes = self.get_patch(start_commit_id, end_commit_id)
        # We need to search through the indexes to find the diff for our file_path
        patch = None
        for diff in diff_indexes:
            #print(f"'{file_path}' '{diff.a_path}' '{diff.b_path}' ")
            if diff.a_path == file_path or diff.b_path == file_path:
                #print("Found match")
                patch = diff.diff
                patch = patch.decode('utf-8')
                break
        return patch

    def get_branch_differences(self, start_branch, end_branch):
        changed_files = {}

        start_commit_id = self.get_commit_from_branch(start_branch)
        end_commit_id = self.get_commit_from_branch(end_branch)
        return self.get_patch(start_commit_id, end_commit_id)
    
    def get_file_contents(self, file_path, commit_id="HEAD"):
        commit = self.repo.commit(commit_id)
        tree = self.repo.tree(commit)
        blob = tree[file_path]
        return blob.data_stream.read().decode()
    
    def get_commits_for_file(self, file_path, max_count=10):
        commits_for_file_generator = self.repo.iter_commits(all=True, max_count=max_count, paths=file_path)
        commits_for_file = list(commits_for_file_generator)
        return commits_for_file
    
    def get_commit_from_branch(self, branch_name):
        return self.repo.heads[branch_name].commit


In [8]:
gd = GitDiff(example_git_path)

contents = gd.get_file_contents("src/main/java/com/redhat/coolstore/service/CatalogService.java")
#print(f"Contents of CatalogService.java at HEAD:  \n{contents}\n")

#Remember that the original state of the file is in the second commit
#The current state of the file is in the first commit
patch = gd.get_patch_for_file(commits_for_file[1].hexsha, commits_for_file[0].hexsha, example_file)

if patch:
    #sPatch = patch.decode('utf-8')
    sPatch = patch
    print(f'Patch for {example_file} between {commits_for_file[1].hexsha} and {commits_for_file[0].hexsha}:\n{sPatch}')
    changed_lines = sPatch.split('\n')[2:]
    additions = sum(1 for line in changed_lines if line.startswith('+'))
    deletions = sum(1 for line in changed_lines if line.startswith('-'))
    print(f'Changes: {additions} additions, {deletions} deletions')
    # For the example we chose, there are only 2 commits
    # so we can assume that the diff from latest commit and 
    # first commit will match from our diff and what is in the 
    # assertion below
    assert additions == commits_for_file[0].stats.files[example_file]['insertions']
    assert deletions == commits_for_file[0].stats.files[example_file]['deletions']
else:
    print(f'No changes in {example_file} between {commits_for_file[1].hexsha} and {commits_for_file[0].hexsha}')


Patch for src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java between 6322db01226ef4a561244ce90a5df946b163f7ba and 9fd5a29585234b32cdd81cea71d71408760c489b:
@@ -1,22 +1,20 @@
 package com.redhat.coolstore.service;
 
-import javax.ejb.ActivationConfigProperty;
-import javax.ejb.MessageDriven;
-import javax.inject.Inject;
-import javax.jms.JMSException;
-import javax.jms.Message;
-import javax.jms.MessageListener;
-import javax.jms.TextMessage;
+// import javax.ejb.ActivationConfigProperty;
+// import javax.ejb.MessageDriven;
+import jakarta.inject.Inject;
+import jakarta.transaction.Transactional;
+
+import org.eclipse.microprofile.reactive.messaging.Incoming;
 
 import com.redhat.coolstore.model.Order;
 import com.redhat.coolstore.utils.Transformers;
-import weblogic.i18n.logging.NonCatalogLogger;
 
-@MessageDriven(name = "OrderServiceMDB", activationConfig = {
-	@ActivationConfigProperty(propertyName = "destinationLookup", propertyValue = "topic/orders"),
-	@ActivationConf

Showing how to get a patch/diff for a file between 2 branches

In [9]:
commit_main = gd.get_commit_from_branch('main')
commit_quarkus = gd.get_commit_from_branch('quarkus-migration')

patch = gd.get_patch_for_file(commit_main.hexsha, commit_quarkus.hexsha, example_file)
print(f"{patch}")

@@ -1,22 +1,20 @@
 package com.redhat.coolstore.service;
 
-import javax.ejb.ActivationConfigProperty;
-import javax.ejb.MessageDriven;
-import javax.inject.Inject;
-import javax.jms.JMSException;
-import javax.jms.Message;
-import javax.jms.MessageListener;
-import javax.jms.TextMessage;
+// import javax.ejb.ActivationConfigProperty;
+// import javax.ejb.MessageDriven;
+import jakarta.inject.Inject;
+import jakarta.transaction.Transactional;
+
+import org.eclipse.microprofile.reactive.messaging.Incoming;
 
 import com.redhat.coolstore.model.Order;
 import com.redhat.coolstore.utils.Transformers;
-import weblogic.i18n.logging.NonCatalogLogger;
 
-@MessageDriven(name = "OrderServiceMDB", activationConfig = {
-	@ActivationConfigProperty(propertyName = "destinationLookup", propertyValue = "topic/orders"),
-	@ActivationConfigProperty(propertyName = "destinationType", propertyValue = "javax.jms.Topic"),
-	@ActivationConfigProperty(propertyName = "acknowledgeMode", propertyValue = "Auto-ackn

We want to walk through 2 branches and determine every file that has changed and get a summary.

In [10]:
# Diff docs:  https://gitpython.readthedocs.io/en/stable/reference.html#git.diff.Diff

commit_main = gd.get_commit_from_branch('main')
commit_quarkus = gd.get_commit_from_branch('quarkus-migration')

diff_indexes = gd.get_branch_differences('main', 'quarkus-migration')
patches_found = 0
patches_empty = 0
files_with_patches = {}
non_none = 0
#https://gitpython.readthedocs.io/en/stable/reference.html#git.diff.DiffIndex.iter_change_type
# A, D, C, M, R, T, U
# ['A', 'D', 'C', 'M', 'R', 'T', 'U']
# Skipping 'R' to cleanup output for now 
# Error trying to check for 'U', throws an exception
for change_type in ['A', 'D', 'C', 'M', 'T']:
    for di in diff_indexes.iter_change_type(change_type):
        print(f"{change_type} {di.a_path} {di.b_path}")


A None .vscode/settings.json
A None src/main/java/com/redhat/coolstore/rest/User.java
A None src/main/resources/META-INF/resources/keycloak.json
A None src/main/resources/application.properties
A None src/main/resources/application.yml
A None src/main/resources/db/migration/V1_3__CreateSequence.sql
A None src/main/resources/old.properties
D src/main/java/com/redhat/coolstore/utils/DataBaseMigrationStartup.java None
D src/main/java/com/redhat/coolstore/utils/Producers.java None
D src/main/java/com/redhat/coolstore/utils/StartupListener.java None
D src/main/resources/META-INF/persistence.xml None
D src/main/webapp/keycloak.json None
M pom.xml pom.xml
M src/main/java/com/redhat/coolstore/model/CatalogItemEntity.java src/main/java/com/redhat/coolstore/model/CatalogItemEntity.java
M src/main/java/com/redhat/coolstore/model/InventoryEntity.java src/main/java/com/redhat/coolstore/model/InventoryEntity.java
M src/main/java/com/redhat/coolstore/model/Order.java src/main/java/com/redhat/coolstor

In [1]:
from git import Repo

file_path = "backend-v1/src/main/java/org/jboss/examples/ticketmonster/util/Resources.java"

tmRepo = Repo("../data/ticket-monster") 
start_commit = tmRepo.commit("master")
tree = tmRepo.tree(start_commit)
blob = tree[file_path]
print(blob.data_stream.read().decode())



NameError: name 'Repo' is not defined